# Experimento 6 - DenseNet
***
- Conjunto de Dados: NIH
- Analisando o treinamento da rede com multiclassificadores
- Arquitetura base: DenseNet

### Importação dos Pacotes

In [2]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.applications.densenet import DenseNet121
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [3]:
# importando os dataframes dos dados de treinamento, validação e teste
train_df = pd.read_csv('/content/drive/MyDrive/train_df.csv', sep = ',', index_col=  0)
validation_df = pd.read_csv('/content/drive/MyDrive/validation_df.csv', sep = ',', index_col=  0)
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv' , sep = ',', index_col=  0)

In [4]:
# visualizando como é a organização do dataframe pré-processado
train_df.head()

,Image Index,finding_labels,labels
87745,/content/drive/MyDrive/images-nih1/images/0000...,Consolidation,1
82970,/content/drive/MyDrive/images-nih1/images/0000...,No Finding,0
51704,/content/drive/MyDrive/images-nih3/images/0001...,No Finding,0
38340,/content/drive/MyDrive/images-nih3/images/0001...,No Finding,0
71153,/content/drive/MyDrive/images-nih4/images/0002...,Infiltration,1


In [5]:
# dicionário com o balancemanto de imagens por classe dos dados de treinamento
class_weights = {0: 3902, 1: 1224, 2: 578, 3: 3645, 4: 818, 5: 671,
                 6: 8770, 7: 55622, 8: 2497, 9: 1028, 10: 295, 11: 2024}

In [6]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255., rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      x_col = 'Image Index',
                                                      y_col = 'finding_labels',
                                                      batch_size = 512,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                    dataframe = validation_df,
                                                    x_col = 'Image Index',
                                                    y_col = 'finding_labels',
                                                    batch_size = 256,
                                                    seed = 42,
                                                    shuffle = True,
                                                    class_mode = 'categorical',
                                                    target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  x_col = 'Image Index',
                                                  y_col = 'finding_labels',
                                                  batch_size = 256,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 81070 validated image filenames belonging to 12 classes.
Found 4268 validated image filenames belonging to 12 classes.
Found 2640 validated image filenames belonging to 12 classes.


### Preparando a rede neural convolucional

In [9]:
base_model = DenseNet121(weights='imagenet', include_top=False)

# pegando a última camada da DenseNet
x = base_model.output

# adicionando uma camada de Global Average Pooling
x = layers.GlobalAveragePooling2D()(x)

# adicionando a camada de saída da rede 
predictions = layers.Dense(units = 12, activation = 'softmax')(x)

# concatenando a rede importada com a rede definida (ajuste fino)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in model.layers:
  print(layer.name)

In [ ]:
# definindo as flags iniciais  
model.trainable = True
set_trainable = False

# para a arquitetura DenseNet, a rede será retreinada a partir da camada 'conv5_block1_1_conv'
for layer in model.layers:
    if layer.name == 'conv5_block1_1_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/densenet/transferlearning_weights_v3.0.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, 
                             mode = 'max')

In [ ]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 81072 // 512, 
                              validation_data = valid_generator, validation_steps = 4268 // 256,
                              callbacks = callbacks, epochs = 10, class_weight = class_weights)

In [13]:
train_df['finding_labels'].value_counts()

No Finding            55622
Infiltration           8770
Atelectasis            3902
Effusion               3645
Nodule                 2497
Pneumothorax           2024
Consolidation          1224
Pleural_Thickening     1028
Emphysema               818
Fibrosis                671
Edema                   578
Pneumonia               295
Name: finding_labels, dtype: int64

In [14]:
train_generator.class_indices

{'Atelectasis': 0,
 'Consolidation': 1,
 'Edema': 2,
 'Effusion': 3,
 'Emphysema': 4,
 'Fibrosis': 5,
 'Infiltration': 6,
 'No Finding': 7,
 'Nodule': 8,
 'Pleural_Thickening': 9,
 'Pneumonia': 10,
 'Pneumothorax': 11}